In [ ]:
!pip install -r requirements.txt

In [3]:
!python mineru_pipeline_all.py

^C


In [5]:
!python fitz__block_chunk.py

📄 正在处理: 中恒电气-公司研究报告-HVDC方案领头羊AI浪潮下迎新机-25071124页.pdf
📄 正在处理: 亚翔集成-公司研究报告-迎接海外业务重估-25071324页.pdf
📄 正在处理: 亿欧中国企业出海沙特季度研究报告-AI专题2025Q267页.pdf
📄 正在处理: 伊利专业乳品2022中国现制茶饮渠道消费者与行业趋势报告40页.pdf
📄 正在处理: 伊利股份-产业周期叠加内生动力业绩增速向上-21093038页.pdf
📄 正在处理: 伊利股份-产品升级叠加费用率下降盈利能力持续提升-22041256页.pdf
📄 正在处理: 伊利股份-公司深度报告王者荣耀行稳致远-22021459页.pdf
📄 正在处理: 伊利股份-公司研究专题报告高股息铸盾景气度为矛当前重点推荐-23051519页.pdf
📄 正在处理: 伊利股份-公司研究报告-平台化的乳企龙头引领行业高质量转型-25071638页.pdf
📄 正在处理: 伊利股份-公司研究报告黑夜终将过去把握高股息低估值乳品龙头机会-24110830页.pdf
📄 正在处理: 伊利股份-大象起舞龙头远航-2020072627页.pdf
📄 正在处理: 伊利股份-格局之变提供发展机遇内生外延打造第二曲线-22052743页.pdf
📄 正在处理: 伊利股份-王者之路扶摇而上-21083136页.pdf
📄 正在处理: 伊利股份-长期规划核心逻辑清晰十年问鼎全球乳业新高点1-21070625页.pdf
📄 正在处理: 伊利股份内蒙古伊利实业集团股份有限公司2024年年度报告.pdf
📄 正在处理: 伊利股份内蒙古伊利实业集团股份有限公司2024年第三季度报告.pdf
📄 正在处理: 伊利股份内蒙古伊利实业集团股份有限公司2025年第一季度报告.pdf
📄 正在处理: 伊利股份多元优势铸造乳企龙头未来战略目标平稳推进42页.pdf
📄 正在处理: 伟星股份-公司首次覆盖服饰辅料龙头柔性制造打造核心竞争力-25071313页.pdf
📄 正在处理: 传音控股-公司研究报告-非洲手机领军者多元化布局品类扩张生态链延伸打开成长空间-25071636页.pdf
📄 正在处理: 儒竞科技-公司研究报告-热泵控制器领导者新能车热管理及自动化构筑新增长曲线-25071036页.pdf
📄 正在处理: 光明肉业-公司

In [13]:
# Rerank probe
from rag_from_page_chunks import SiliconFlowReranker
reranker = SiliconFlowReranker()

docs=[{"content":"图1：Q3开始下降。"}, {"content":"无关介绍"}]
ranked = reranker.rerank("哪一季度开始下降？", docs)
print("rerank OK:", [(d["rerank_score"], d["content"][:10]) for d in ranked])



rerank OK: [(0.9817707, '图1：Q3开始下降。'), (3.295573e-05, '无关介绍')]


In [6]:
import os, json, time, subprocess
from pathlib import Path

# --- Batch config ---
start = 0          # inclusive
step  = 60         # per-batch size
end   = 806         # exclusive
max_retries = 10

# --- RAG behavior flags (your SimpleRAG reads these) ---
env = os.environ.copy()
env.update({
    "LOCK_SOURCE":  "1",   # force filename/page to top evidence
    "STRICT_RERANK":"0",   # set "1" to fail fast if rerank API hiccups
    "DEBUG_RAG":    "1",   # include telemetry in outputs
    "PAGE_OFFSET":  "0",
    "LIMIT_PER_FILE": "16",
    "BM25_TOPN":"35",
    "RRF_K":"75",
})
os.environ.update(env)

parts_dir = Path("./parts")
parts_dir.mkdir(exist_ok=True)

while start < end:
    retries = 0
    while retries < max_retries:
        cmd = ["python", "rag_from_page_chunks.py", str(start), str(step)]
        print(f"\n=== 正在运行批次 {start} ~ {start+step-1} (尝试 {retries+1}/{max_retries}) ===")

        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                                text=True, env=env)

        success = False
        for line in proc.stdout:
            print(line, end="")
            if "已输出本段结构化结果" in line:
                success = True

        proc.wait()

        if success:
            print(f"✅ 批次 {start} 成功")

            # (Optional) Quick sanity peek: read telemetry from the expected part file
            part_tag = f"{start:06d}_{min(start+step-1, end-1):06d}"
            part_path = parts_dir / f"part_{part_tag}.json"
            if part_path.exists():
                try:
                    data = json.loads(part_path.read_text(encoding="utf-8"))
                    # print a tiny summary for first few rows if debug present
                    for i, row in enumerate(data[:3]):
                        dbg = row.get("debug") or {}
                        print(f"  • [{i}] path={dbg.get('path')} cand={dbg.get('cand_n')} "
                              f"ranked={dbg.get('ranked_n')} pagevote={dbg.get('page_vote_n')} "
                              f"final={dbg.get('final_n')} lock={env['LOCK_SOURCE']}")
                except Exception as e:
                    print(f"  (无法读取调试信息: {e})")
            start += step
            break
        else:
            retries += 1
            sleep_s = min(60, 2 ** retries)  # exponential backoff
            print(f"❌ 批次 {start} 失败，{sleep_s}s 后重试（已重试 {retries}/{max_retries}）…")
            time.sleep(sleep_s)

    if retries >= max_retries:
        print(f"⚠️ 批次 {start} 连续失败 {max_retries} 次，跳过该批次")
        start += step



=== 正在运行批次 0 ~ 59 (尝试 1/10) ===
加载所有页chunk...
共加载 10053 个chunk
生成嵌入...

Embedding: 100%|██████████| 315/315 [04:21<00:00,  1.20batch/s]
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.659 seconds.
Prefix dict has been built successfully.
RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}. 等待10秒后重试（第1次）...
RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}. 等待10秒后重试（第1次）...
RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}. 等待10秒后重试（第1次）...
RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limiting. Details: TPM limit reached.', 'data': None}. 等待10秒后重试（第1次）...
RateLimitError: Error code: 429 - {'message': 'Request was rejected due to rate limit

In [1]:
!python mergepart.py

✅ 合并完成: ./rag_top1_pred.json （共 806 条）
